In [20]:
import cvxpy as cp
import numpy as np
import os
import math
import pandas as pd
import datetime as dt
from gurobipy import Model, GRB, quicksum
import gurobipy as gp
import warnings
import time
from scipy.stats import norm

### 主问题求解(cut的生成)

In [27]:
def Allocation_2D(
        iter, NP, N_Bus, ESS_candidate, R_bounds, C_bounds, obj_2nd, lambda_2nd, mu_2nd, 
        previous_rating, previous_cap, Fixed_cost, Power_rating_cost, Energy_capacity_cost, C_rate=1):

    print("start solving master problem...")

    # Unfolding data:
    R_min = R_bounds[0]
    R_max = R_bounds[1]
    C_min = C_bounds[0]
    C_max = C_bounds[1]

    # Adding variables for MILP
    U = cp.Variable((N_Bus), boolean=True) # Location for ESS solutions
    Cap_U = cp.Variable(N_Bus)  # Continuous variable representing the maximum energy storage capacity at each node.
    Rating_U = cp.Variable(N_Bus)  # Continuous variable representing the maximum energy storage rating at each node.
    alpha = cp.Variable() # using for benders decomposition

    #Values that need to be calculated
    non_candidate = np.ones(N_Bus)
    non_candidate[ESS_candidate] = 0 # Usefull to constraint the location of ESS storage system to only candidate nodes

    # Constraints: 
    constraints = []
    # Physical limits on power and capacity
    constraints += [Rating_U >= cp.multiply(R_min, U), Rating_U <= cp.multiply(R_max, U)] # Constraint rating to max and min values
    constraints += [Cap_U >= cp.multiply(C_min, U), Cap_U <= cp.multiply(C_max, U)] # Constraint capacity to max and min values
    constraints.append(cp.multiply(non_candidate,U) == np.zeros(N_Bus))
    constraints.append(C_rate * Cap_U >= Rating_U)

    # Benders decompositions 
    bdcut = []  # Multi-cut
    bdcut2 = []  # Single-cut

    for k in range(iter):
        for sc in range(NP):
            bdcut.append(alpha >= obj_2nd[sc, k] + 
                        cp.sum(lambda_2nd[sc, :, :, k], axis=1) @ (Rating_U - previous_rating[:, k]) + 
                        cp.sum(mu_2nd[sc, :, :, k], axis=1) @ (Cap_U - previous_cap[:, k]))

        bdcut2.append(alpha >= cp.sum(obj_2nd[:, k]) + 
                    np.sum(np.sum(lambda_2nd[:, :, :, k], axis=2), axis=0).T @ (Rating_U - previous_rating[:, k]) + 
                    np.sum(np.sum(mu_2nd[:, :, :, k], axis=2), axis=0).T @ (Cap_U - previous_cap[:, k]))

    constraints += bdcut2 + [alpha >= 0] + bdcut

    # Objective function
    objective = (
        cp.multiply(Fixed_cost, cp.sum(U)) + 
        cp.multiply(Power_rating_cost, cp.sum(Rating_U)) + 
        cp.multiply(Energy_capacity_cost, cp.sum(Cap_U)) + 
        alpha
    )

    # Solve
    problem = cp.Problem(cp.Minimize(objective), constraints)
    problem.solve(solver=cp.MOSEK)

    Investment = (
        Fixed_cost * np.sum(U.value) + 
        Power_rating_cost * np.sum(Rating_U.value) + 
        Energy_capacity_cost * np.sum(Cap_U.value)
    )
    
    print("finish solving master problem")

    return problem.value, Investment, U.value, Rating_U.value, Cap_U.value, alpha.value

### Incidence_matrices计算

In [28]:
def Incidence_matrices(num_nodes, num_lines, sending_end, receiving_end):
    """
    Create incidence matrices for network topology
    """
    A_plus = np.zeros((num_lines,num_nodes)) # A+ matrix (num_nodes x num_lines)
    for l in range(num_lines):
        A_plus[l,int(sending_end[l])] = 1
        A_plus[l,int(receiving_end[l])] = -1

    A_minus = np.zeros((num_lines,num_nodes)) # A- matrix (num_nodes x num_lines)
    for l in range(num_lines):
        A_minus[l,int(sending_end[l])] = 0
        A_minus[l,int(receiving_end[l])] = -1

    A_plus = np.transpose(A_plus)
    A_minus = np.transpose(A_minus)

    return A_plus, A_minus

### average_cut函数

In [29]:
def average_cut_full(NP, NB, NT, R_sample, obj_val_tilde, lambda_val_tilde, mu_val_tilde):
    """
    Expand sampled dual values and objective values to full arrays,
    using mean for non-sampled scenarios.
    """
    obj_full = np.zeros(NP)
    lambda_full = np.zeros((NP, NB, NT))
    mu_full = np.zeros((NP, NB, NT))

    # Fill sampled
    obj_full[R_sample] = obj_val_tilde
    lambda_full[R_sample, :, :] = lambda_val_tilde
    mu_full[R_sample, :, :] = mu_val_tilde

    # Compute means
    obj_mean = np.mean(obj_val_tilde)
    lambda_mean = np.mean(lambda_val_tilde, axis=0)  # shape (NB, NT)
    mu_mean = np.mean(mu_val_tilde, axis=0)

    # Fill non-sampled with mean
    not_R_sample = np.setdiff1d(np.arange(NP), R_sample)
    obj_full[not_R_sample] = obj_mean
    lambda_full[not_R_sample, :, :] = lambda_mean
    mu_full[not_R_sample, :, :] = mu_mean

    return obj_full, lambda_full, mu_full

### 子问题求解(grid operation)

#### single scenario

In [ ]:
def compute_SOC_ACOPF(sc, NT, baseMVA, N_bus, N_line, Yp, sending_node, receiving_node,
                      R_l, X_l, B_l, Pd, Qd, pn_bound, qn_bound, v_bound, G_n, B_n,
                      K_l, a, b, c, ESS_soc0, ESS_cha_bound, ESS_dis_bound, ESS_soc_bound, Pn_solar_bound, freq_scenario):

    # Extract time-dependent indices
    idx_PV_sc = [np.where(Pn_solar_bound[sc, 1, :, time] > 0)[0].astype(int) for time in range(NT)]
    Yp_scenario = Yp * freq_scenario

    try:
        print(f"Starting scenario {sc}", flush=True)
        # Call the SOC_ACOPF_2D_alocation function
        # solving sub-problem to get ub and dual solutions
        cost, _, _, _, _, _, _, _, _, _, _, _, _, _, _, lambda_aloc, mu_aloc = SOC_ACOPF_2D_alocation(
            baseMVA, NT, N_bus, N_line, Yp_scenario[sc], sending_node, receiving_node, idx_PV_sc,
            R_l, X_l, B_l, Pd[sc], Qd[sc],
            pn_bound[sc], qn_bound, v_bound,
            G_n, B_n, K_l,
            a, b, c,
            ESS_soc0, ESS_cha_bound, ESS_dis_bound, ESS_soc_bound)

        print(f"Finished scenario {sc}: Cost = {cost}", flush=True)
        # Return the results for storage
        return cost, lambda_aloc, mu_aloc

    except Exception as e:
        print(f"Error in compute_SOC_ACOPF for scenario {sc}: {e}", flush=True)
        raise

In [ ]:
def SOC_ACOPF_2D_alocation(baseMVA, NT, num_nodes, num_lines, Yp, sending_node, receiving_node, IndPV,
               R_l, X_l, B_l, Pd, Qd, pn_bound, qn_bound, v_bound, G_n, B_n, K_l, quad_cost, lin_cost, const_cost,
               ESS_soc0, ESS_cha_bound, ESS_dis_bound, ESS_soc_bound, 
               theta_n_min=-1, theta_n_max=-1, theta_l_min=-1, theta_l_max=-1, eta_dis = 1, eta_cha = 1):

    ######################################################
    """This model is very usefull since it allows to compute the optimal power flow for any grid topology."""
    ######################################################
    """Inputs"""
    # baseMVA : size(1) : Power base in MVA
    # NT : size(1) : Number of time steps
    # num_nodes : size(1) : Number of buses in the grid (NB)
    # num_lines : size(1) : Number of lines in the grid (NL)
    # Yp : size(1) : Planning period
    # sending_node : size(NB) : array with the sending node with number from 0 to N_bus-1 for line l 
    # sending_node : size(NL) : array with the receiving node with number from 0 to N_bus-1 for line l
    # IndPV : Indexes of PV : List where are the PV
    """All values in p.u. except the cost functions [CHF/MW]"""
    # quad_cost : Quadratic cost coefficient
    # lin_cost : Linear cost coefficient
    # const_cost : Constant cost coefficient
    # R_l, X_l, B_l, K_l : size(NL,NT) : r, x, b, ampacity limit for each line at each time step for each line
    # p_d, q_d, G_n, B_n : size(NB,NT) : active power, reactive power, g and b, at each time step for each node
    # pn_bound, qn_bound, v_bound : size(2,NB,NT) : active power, reactive power and voltage magnitude bounds at each time step for each node
    # quad_cost, lin_cost, const_cost : size(NB,NT) : cost of each buses.
    # ESS_soc0, ESS_cha_bound, ESS_dis_bound, ESS_soc_bound : size(2,NB,NT) : ESS initial state and bounds for rating and capacity
    ######################################################
    """Output"""
    # problem : size(1) : Total cost of operation
    # p_n, q_n, v_n : size(NB,NT) : active power injection, reactive power injection and voltage magnitude at each time step for each node
    # p_sl, q_sl, p_ol, q_ol, K_ol : size(NL,NT) : active and reactive power flow on lines, active and reactive power flow losses on lines, ampacity losses
    # theta_n, theta_l : size(NB or NL,NT) : angles on buses and lines
    # ESS_soc, ESS_cha, ESS_dis, q_ESS : size(NB,NT) : ESS operation variables 
    # lambda_, mu_ : size(NB,NT) : Dual values of ESS constraints on rating and capacity
    ######################################################
    
    """Initialisation"""
    A_plus, A_minus = Incidence_matrices(num_nodes, num_lines, sending_node, receiving_node)

    # Unfolding nodes data
    p_n_min = pn_bound[0]  # Minimum active power generation at each node
    p_n_max = pn_bound[1]  # Maximum active power generation at each node
    q_n_min = qn_bound[0]  # Minimum reactive power generation at each node
    q_n_max = qn_bound[1]  # Maximum reactive power generation at each node
    V_min = v_bound[0]**2  # Minimum voltage squared at each node
    V_max = v_bound[1]**2  # Maximum voltage squared at each node
    ESS_cha_min = ESS_cha_bound[0] # Minimum charging rate at each node 
    ESS_cha_max = ESS_cha_bound[1] # Maximum charging rate at each node
    ESS_dis_min = ESS_dis_bound[0] # Minimum discharging rate at each node 
    ESS_dis_max = ESS_dis_bound[1] # Maximum discharging rate at each node
    ESS_soc_min = ESS_soc_bound[0] # Minimum state of charge at each node 
    ESS_soc_max = ESS_soc_bound[1] # Maximum state of charge at each node

    # theta_n min and max
    if theta_n_min == -1 : theta_n_min = - np.pi / 2 * np.ones((num_nodes,NT))  # Minimum bus angle 
    if theta_n_max == -1 : theta_n_max = np.pi / 2 * np.ones((num_nodes,NT))  # Maximum bus angle

    # theta_l min and max
    if theta_l_min == -1 : theta_l_min = - np.pi / 2 * np.ones((num_lines,NT))  # Minimum line angle (from relaxation assumption)
    if theta_l_max == -1 : theta_l_max = np.pi / 2 * np.ones((num_lines,NT))  # Maximum line angle (from relaxation assumption)

    # for q_ESS
    lin = 2 # from data
    xx = np.linspace(0, 1/2 * np.pi, lin + 1)
    slope = np.zeros(lin)
    offset = np.zeros(lin)
    for i in range(lin):
        slope[i]=(np.sin(xx[i+1])-np.sin(xx[i]))/(np.cos(xx[i+1])-np.cos(xx[i]))
        offset[i]=(np.sin(xx[i])*np.cos(xx[i+1])-np.sin(xx[i+1])*np.cos(xx[i]))/(np.cos(xx[i+1])-np.cos(xx[i]))

    ######################################################
    """Variables"""
    p_n = cp.Variable((num_nodes,NT))  # Active power at node n
    p_curtailment = cp.Variable((num_nodes, NT), nonneg=True)
    p_slack = p_n[0, :]
    p_imp = cp.pos(p_slack)
    p_exp = cp.pos(-p_slack)
    q_n = cp.Variable((num_nodes,NT))  # Reactive power at node n
    V_n = cp.Variable((num_nodes, NT))  # Voltage magnitude squared at node n
    theta_n = cp.Variable((num_nodes,NT))  # Voltage angles at node n
    p_sl = cp.Variable((num_lines,NT))  # Active power at sending end of line l
    q_sl = cp.Variable((num_lines,NT))  # Reactive power at sending end of line l
    p_ol = cp.Variable((num_lines,NT))  # Active power losses on line l
    q_ol = cp.Variable((num_lines,NT))  # Reactive power losses on line l
    K_ol = cp.Variable((num_lines,NT))  # Branch Equivalent ampacity constraint on line l
    theta_l = cp.Variable((num_lines,NT))  # Voltage angles at line l

    ESS_cha = cp.Variable((num_nodes,NT)) 
    ESS_dis = cp.Variable((num_nodes,NT))
    ESS_soc = cp.Variable((num_nodes,NT))
    q_ESS = cp.Variable((num_nodes,NT))
    
    # Variables for allocation
    Cmax = cp.Variable((num_nodes,NT))
    Rmax = cp.Variable((num_nodes,NT))

    # Create the Incidence Matrices used in the sending end and receiving end voltage
    Inc_sending_cvx = np.zeros((num_lines, num_nodes))
    Inc_receiving_cvx = np.zeros((num_lines, num_nodes))
    for l in range(num_lines):
        Inc_sending_cvx[l, sending_node[l]] = 1
        Inc_receiving_cvx[l, receiving_node[l]] = 1

    ######################################################
    """ Constraints"""
    constraints = []
    objective = 0
    
    ### Bus constraints ###

    # Voltage Magnitude bounds (1k)
    constraints.append(V_n >= V_min)
    constraints.append(V_n <= V_max)

    # Node angle bounds (1m)
    constraints.append(theta_n >= theta_n_min)
    constraints.append(theta_n <= theta_n_max)

    # Active power bounds (1n)
    constraints.append(p_n >= p_n_min)
    constraints.append(p_n <= p_n_max)

    # for time in range(NT):
    #     if len(IndPV[time])>0:
    #         constraints.append(p_n[IndPV[time],:] == p_n_max[IndPV[time],:]) # enforces the power injection to be equal to PV production

    for time in range(NT):
        if len(IndPV[time]) > 0:
            # p_n + p_curtailment = p_n_max
            constraints.append(p_n[IndPV[time], :] + p_curtailment[IndPV[time], :] == p_n_max[IndPV[time], :])

    # Reactive power bounds (1o)
    constraints.append(q_n >= q_n_min)
    constraints.append(q_n <= q_n_max)

    # Alocation related constraints to get the dual variables
    constraint_capacity = Cmax == ESS_soc_max
    constraint_rating = Rmax == ESS_cha_max
    constraints += [constraint_capacity, constraint_rating]

    # ESS charging and discharging rate bounds
    constraints.append(ESS_cha >= ESS_cha_min)
    constraints.append(ESS_cha <= Rmax)
    constraints.append(ESS_dis >= ESS_dis_min)
    constraints.append(ESS_dis <= Rmax)

    # ESS SOC bounds
    constraints.append(ESS_soc >= ESS_soc_min)
    constraints.append(ESS_soc <= Cmax)

    # Linking time steps for ESS (excluding the last time step)
    constraints.append(ESS_soc[:, 1:] == ESS_soc[:, :-1] + ESS_cha[:, :-1] - ESS_dis[:, :-1])
    constraints.append(ESS_soc0 == ESS_soc[:,-1] + ESS_cha[:,-1] - ESS_dis[:,-1]) #last timestep to reset battery charge for next day

    # Initializing ESS SOC for the first time step
    constraints.append(ESS_soc[:, 0] == ESS_soc0)
    constraints.append(ESS_soc[:,-1] == ESS_soc0)
    constraints.append(cp.sum(ESS_cha,axis=1) == cp.sum(ESS_dis,axis=1))

    # Battery aging constraints --> battery has to do at maximum 1.1 cycles per day
    constraints.append(cp.sum(cp.abs(ESS_cha-ESS_dis),axis=1) <= 2*1.1*Cmax[:,0])

    # ESS reactive power computation and bounds
    for i in range(lin):
        constraints.append(q_ESS <= slope[i] * (ESS_cha - ESS_dis) + offset[i] * Rmax)
        constraints.append(q_ESS <= -slope[i] * (ESS_cha - ESS_dis) + offset[i] * Rmax)
        constraints.append(q_ESS >= slope[i] * (ESS_cha - ESS_dis) - offset[i] * Rmax)
        constraints.append(q_ESS >= -slope[i] * (ESS_cha - ESS_dis) - offset[i] * Rmax)

    # Active Power Balance (1b)
    constraints.append(p_n + ESS_dis - Pd - ESS_cha == A_plus @ p_sl - A_minus @ p_ol + cp.multiply(G_n, V_n))

    # Reactive Power Balance (1c)
    constraints.append(q_n - q_ESS - Qd == A_plus @ q_sl - A_minus @ q_ol - cp.multiply(B_n, V_n))

    ### line constraints ###

    # Line angle bounds (1l):
    constraints.append(theta_l >= theta_l_min)
    constraints.append(theta_l <= theta_l_max)

    # Voltage drop constraint (1d):
    constraints.append(Inc_sending_cvx @ V_n - Inc_receiving_cvx @ V_n== 2 * cp.multiply(R_l, p_sl) + 2 * cp.multiply(X_l, q_sl) - cp.multiply(R_l, p_ol) - cp.multiply(X_l, q_ol))
    
    # Conic active and reactive power losses constraint (2b):
    constraints.append(K_ol == cp.multiply((K_l - cp.multiply(Inc_sending_cvx @ V_n, B_l**2) + 2 * cp.multiply(q_sl, B_l)), X_l))
    constraints.append(K_ol >= q_ol)

    # Power loss constraint (2c):
    constraints.append(cp.multiply(p_ol,X_l) == cp.multiply(q_ol,R_l))

    # Line angle constraint (1h):
    constraints.append(theta_l == Inc_sending_cvx @ theta_n - Inc_receiving_cvx @ theta_n)

    # Linearized angle constraint (2d):
    constraints.append(theta_l == cp.multiply(X_l,p_sl) - cp.multiply(R_l,q_sl))

    # Constraints that requiere a loop because of dimensionality limit of cp.norm().
    for time in range(NT):
        for l in range(num_lines):
            # Conic active and reactive power losses constraint (2b): (rest of ineq)
            constraints.append(
                cp.norm(
                    cp.vstack([
                        2 *np.sqrt(X_l[l,time])* cp.vstack([p_sl[l,time], q_sl[l,time]]),
                        cp.reshape(q_ol[l,time] - V_n[sending_node[l],time], (1, 1))
                    ]),2
                ) <= q_ol[l,time] + V_n[sending_node[l],time]
            )

            # Feasibility solution recovery equation (4g):
            constraints.append(
                V_n[sending_node[l],time] + V_n[receiving_node[l],time] >= cp.norm(
                    cp.vstack([
                        2*theta_l[l,time]/np.sin(theta_l_max[l,time]), 
                        V_n[sending_node[l],time] - V_n[receiving_node[l],time]
                    ]), 2)
            )
    

    #####################################################################
    """Objective Function""" 
    curtailment_cost = 26
    price_imp = [620, 620, 620, 620, 620, 620, 620, 890, 890, 890, 890, 890, 890, 890, 890, 890, 890, 890, 890, 890, 890, 620, 620, 620]
    objective = cp.Minimize(
        Yp * 365 * (cp.sum(cp.multiply(quad_cost, cp.square(p_n * baseMVA)) 
                        + cp.multiply(lin_cost, p_n * baseMVA) 
                       + const_cost)) 
        + Yp * 2910 * cp.sum(ESS_cha + ESS_dis) * baseMVA
        + cp.sum(p_ol) * 100 * 365 * baseMVA * Yp
        + cp.sum(cp.multiply(p_imp, price_imp)) * 365 * baseMVA * Yp
        + cp.sum(p_exp) * 100 *365 *baseMVA * Yp
        + cp.sum(p_curtailment * baseMVA) * curtailment_cost * Yp * 365
    )

    # Defining the optimization problem
    problem = cp.Problem(objective, constraints)
    #####################################################################

    #####################################################################
    # Solve the problem
    problem.solve(solver=cp.MOSEK)

    # Dual values :
    lambda_ = constraint_rating.dual_value # Dual value for the constraint related to ESS maximal rating
    mu_ = constraint_capacity.dual_value # Dual value for the constraint related to ESS maximal capacity

    return problem.value, p_n.value, q_n.value, np.sqrt(V_n.value), p_sl.value, q_sl.value, p_ol.value, q_ol.value, K_ol.value, theta_n.value, theta_l.value, ESS_soc.value, ESS_cha.value, ESS_dis.value, q_ESS.value, lambda_, mu_

#### all scenario

In [ ]:
def run_SOC_ACOPF_for_samples(NP, R_sample, NT, baseMVA, N_bus, N_line, Yp, sending_node, receiving_node,
                              R_l, X_l, B_l, Pd, Qd, pn_bound, qn_bound, v_bound, G_n, B_n,
                              K_l, a, b, c, ESS_soc0, ESS_cha_bound, ESS_dis_bound, ESS_soc_bound,
                              Pn_solar_bound, freq_scenario):
    lambda_list = []
    mu_list = []
    obj_list = []

    for sc in R_sample:
        obj_val, lambda_val, mu_val = compute_SOC_ACOPF(
            sc, NT, baseMVA, N_bus, N_line, Yp, sending_node, receiving_node,
            R_l, X_l, B_l, Pd, Qd, pn_bound, qn_bound, v_bound, G_n, B_n,
            K_l, a, b, c, ESS_soc0, ESS_cha_bound, ESS_dis_bound,
            ESS_soc_bound, Pn_solar_bound, freq_scenario
        )
        obj_list.append(obj_val)
        lambda_list.append(lambda_val)
        mu_list.append(mu_val)

    lambda_array = np.stack(lambda_list, axis=0)
    mu_array = np.stack(mu_list, axis=0)
    obj_array = np.array(obj_list)

    obj_full, lambda_full, mu_full = average_cut_full(NP, N_bus, NT, R_sample, obj_array, lambda_array, mu_array)

    return obj_full, lambda_full, mu_full

### 主函数

In [ ]:
if __name__=="__main__":

    ################################################################ Parameters
    ESS_candidate = np.array([8, 18, 25, 33]) #Candidates nodes
    num_processes = 4 # Number of processes to use (adjust based on your CPUs and memory)
    lim_iter = 100 # Max number of iteration of benders decompositions
    datapath = "Data/" # Change as a function of the loaded folder
    # It is possible to change other vales such as prices and costs detailed later

    ############################################################### Data import
    # Calculate time
    start_time = dt.datetime.now()
    sample_time = start_time

    # data for 33-bus system 
    bus_data = pd.read_csv(datapath+"bus_data.csv")
    # bus_data = bus_data.sort_values(by="BUS_I").reset_index(drop=True) # maybe not usefull, to test later
    # bus_index_to_bus_name = bus_data['BUS_I'].to_dict() # dictionary mapping the index number to bus ID
    # bus_name_to_bus_index = {v: k for k, v in bus_index_to_bus_name.items()}
    # bus_data = bus_data.reset_index().rename(columns = {"index":"grid_node"})

    branch_data = pd.read_csv(datapath+"branch_data.csv")
    generator_data = pd.read_csv(datapath+"generator_data.csv")

    # stat_scenario = pd.read_csv(datapath+"Chaudron_scenarios.csv",delimiter=";")
    # freq_scenario = stat_scenario.dp.to_numpy() / stat_scenario.dp.sum()

    # Processed data to get Pd, Qd and PV production per time step and per Scenario
    scenario_data = pd.read_csv(datapath+"Clean_demand.csv")

    # Used as data cleaning since I had one file with N nodes and the other with N-1 nodes
    # I then assumed that the slack was missing so I ajust values to fit 
    # scenario_data.loc[scenario_data[scenario_data.grid_node>=(bus_data[bus_data.BUS_TYPE==3].grid_node).to_list()[0]].index,"grid_node"] += 1 

    # NP = len(scenario_data.Period.unique()[:-2])
    # N_bus = len(bus_data.grid_node.unique())
    # N_line = len(branch_data)
    # NT = 24
    # baseMVA = generator_data.MBASE.max()

    N_bus = len(bus_data.Grid_node.unique())  # 33-bus
    N_line = len(branch_data)
    NP = len(scenario_data.Scenario.unique())  # 12 scenarios
    NT = 24
    baseMVA = 10  # 1e7VA=10MVA

    freq_scenario = np.ones(NP) / NP

    ############################################################## Setting all costs
    # PV generation cost
    quad_cost_PV = 0
    lin_cost_PV = 26
    const_cost_PV = 0

    # Power cost
    # index_slack = (bus_data[bus_data.BUS_TYPE==3].grid_node).to_list()[0]
    index_slack = [0]
    a_slack = np.zeros(N_bus)
    a_slack[index_slack] = 0
    a_slack = a_slack[:, np.newaxis] * np.ones((1, NT))
    b_slack = np.zeros(N_bus)
    b_slack[index_slack] = 200
    b_slack = b_slack[:, np.newaxis] * np.ones((1, NT))
    c_slack = np.zeros(N_bus)
    c_slack[index_slack] = 0
    c_slack = c_slack[:, np.newaxis] * np.ones((1, NT))

    ############################################################## Data preprocessing
    # creating the dataset for values that depends on time and scenario
    # Pd = np.zeros((NP, N_bus, NT))
    # Pn_solar_bound = np.zeros((NP, 2, N_bus, NT))
    # a_PV = np.zeros((N_bus, NT))
    # b_PV = np.zeros((N_bus, NT))
    # c_PV = np.zeros((N_bus, NT))

    # for sc in scenario_data.Period.unique()[:-2]-1:
    #     mask1 = (scenario_data.Period==sc+1)
    #     for time in scenario_data[mask1].Time.unique()-1:
    #         mask2 = (scenario_data.Time==time)
    #         intermediate_df = pd.merge(scenario_data[mask1 & mask2][["grid_node","Domestic_electricity","PV_production"]],
    #             bus_data[["grid_node"]],
    #             on="grid_node",
    #             how="right").fillna(0)
    #         Pd[sc,:,time] = intermediate_df.Domestic_electricity.to_numpy() /1000 /baseMVA # kWh to MW to p.u. 
    #         Pn_solar_bound[sc,1,:,time] = intermediate_df.PV_production.to_numpy() /1000 /baseMVA # kWh to MW to p.u.
    #         a_PV[:,time] = quad_cost_PV * np.ones(N_bus) # useless in that case since a,b,c constant
    #         b_PV[:,time] = lin_cost_PV * np.ones(N_bus) # useless in that case since a,b,c constant
    #         c_PV[:,time] = const_cost_PV * np.ones(N_bus) # useless in that case since a,b,c constant"""

    Pd = np.zeros((NP, N_bus, NT))
    Qd = np.zeros((NP, N_bus, NT))
    Pn_solar_bound = np.zeros((NP, 2, N_bus, NT))
    index_PV = [16, 31]
    a_PV = np.zeros((N_bus, NT))
    b_PV = np.zeros((N_bus, NT))
    c_PV = np.zeros((N_bus, NT))

    for sc in range(1, NP + 1):
        for time in range(1, NT + 1):
            mask = (scenario_data.Scenario == sc) & (scenario_data.Time == time)
            intermediate_df = pd.merge(
                scenario_data[mask][["Grid_node", "Pd", "Qd", "PV"]],
                pd.DataFrame({"Grid_node": np.arange(1, N_bus + 1)}),
                on="Grid_node",
                how="right"
            ).fillna(0)

            Pd[sc-1, :, time-1] = intermediate_df.Pd.to_numpy() / 1000 / baseMVA  # kW to p.u.
            Qd[sc-1, :, time-1] = intermediate_df.Qd.to_numpy() / 1000 / baseMVA  # kW to p.u.
            Pn_solar_bound[sc-1, 0, :, time-1] = 0
            Pn_solar_bound[sc-1, 1, :, time-1] = intermediate_df.PV.to_numpy() / 1000 / baseMVA  # kW to p.u.

    a_PV[index_PV, :] = quad_cost_PV
    b_PV[index_PV, :] = lin_cost_PV
    c_PV[index_PV, :] = const_cost_PV

    # expanding dataset for all other data that is not time or scenario dependant
    V_base = bus_data.baseKV.max()  # 12.66 kV
    Z_base = (V_base ** 2) / baseMVA  # kV^2 / MVA = 16.02756 Ohm

    # related to bus data
    # q_d = bus_data.Qd.to_numpy() /baseMVA /5 #reactive power demand
    # q_d= q_d[:, np.newaxis] * np.ones((1, NT))
    G_n = bus_data.Gs.to_numpy() * Z_base
    G_n = G_n[:, np.newaxis] * np.ones((1, NT))
    B_n = bus_data.Bs.to_numpy() * Z_base
    B_n = B_n[:, np.newaxis] * np.ones((1, NT))
    v_bound = bus_data[["Vmin","Vmax"]].to_numpy()
    v_bound = v_bound.T[:, :, np.newaxis] * np.ones((1, N_bus, NT))

    # related to generator data
    # pn_bound = np.zeros((N_bus,2))
    # qn_bound = np.zeros((N_bus,2))
    # index_gen = generator_data["GEN_BUS"].map(bus_name_to_bus_index).to_numpy()
    # generator_data["bus_index"] = index_gen
    # for _, row in generator_data.iterrows():
    #     index = row["bus_index"]
    #     pn_bound[index,0] = -row["PMAX"] /baseMVA 
    #     pn_bound[index,1] = row["PMAX"] /baseMVA 
    #     qn_bound[index,0] = row["QMIN"] /baseMVA 
    #     qn_bound[index,1] = row["QMAX"] /baseMVA 
    # pn_bound = pn_bound.T[np.newaxis,:, :, np.newaxis] * np.ones((NP, 1, N_bus, NT))
    # pn_bound += Pn_solar_bound #Add the PV generation to other generation
    # qn_bound = qn_bound.T[:, :, np.newaxis] * np.ones((1, N_bus, NT))

    # if no generator, only contains PV bound
    pn_bound = np.zeros((N_bus, 2))
    qn_bound = np.zeros((N_bus, 2)) 

    slack_Pmax = 1e4  # choose a large headroom in MW
    slack_Pmin = -1e4
    slack_Qmax = 1e4
    slack_Qmin = -1e4
    
    pn_bound[index_slack, 0] = slack_Pmin / baseMVA
    pn_bound[index_slack, 1] = slack_Pmax / baseMVA
    qn_bound[index_slack, 0] = slack_Qmin / baseMVA
    qn_bound[index_slack, 1] = slack_Qmax / baseMVA

    index_gen = (generator_data["GEN_BUS"].astype(int) - 1).values
    # generator_data["bus_index"] = index_gen
    a_gen = np.zeros(N_bus)
    a_gen[index_gen] = [0.12, 0.09]
    a_gen = a_gen[:, np.newaxis] * np.ones((1, NT))
    b_gen = np.zeros(N_bus)
    b_gen[index_gen] = [20, 15]
    b_gen = b_gen[:, np.newaxis] * np.ones((1, NT))
    c_gen = np.zeros(N_bus)
    c_gen[index_gen] = [0, 0]
    c_gen = c_gen[:, np.newaxis] * np.ones((1, NT))

    pn_bound[index_gen, 0] = generator_data["P_min"].values / baseMVA
    pn_bound[index_gen, 1] = generator_data["P_max"].values / baseMVA
    qn_bound[index_gen, 0] = generator_data["Q_min"].values / baseMVA
    qn_bound[index_gen, 1] = generator_data["Q_max"].values / baseMVA

    pn_static = pn_bound.T[np.newaxis, :, :, np.newaxis]  # (1,2,N_bus,1)
    qn_static = qn_bound.T[:, :, np.newaxis]  # (2,N_bus,1)

    # 2) Broadcast to (NP,2,N_bus,NT)
    pn_bound = pn_static * np.ones((NP, 2, N_bus, NT))

    # 3) Finally add the PV bounds
    pn_bound += Pn_solar_bound  # both are now (NP,2,N_bus,NT)
    qn_bound = qn_static * np.ones((2, N_bus, NT))

    # related to branch data
    # sending_node = branch_data["F_BUS"].map(bus_name_to_bus_index).to_numpy().astype(int)
    # receiving_node = branch_data["T_BUS"].map(bus_name_to_bus_index).to_numpy().astype(int)
    sending_node = branch_data["F_BUS"].to_numpy().astype(int) - 1  # converted to 0-based
    receiving_node = branch_data["T_BUS"].to_numpy().astype(int) - 1  # converted to 0-based
    R_l = branch_data["BR_R"].to_numpy() / Z_base
    R_l = R_l[:, np.newaxis] * np.ones((1, NT))
    X_l = branch_data["BR_X"].to_numpy() / Z_base
    X_l = X_l[:, np.newaxis] * np.ones((1, NT))
    B_l = branch_data["BR_B"].to_numpy() * Z_base
    B_l = B_l[:, np.newaxis] * np.ones((1, NT))
    K_l = branch_data["Pup"].to_numpy() / 1000 / baseMVA  # # Ampacity for each line (kW to p.u.)
    K_l = K_l[:, np.newaxis] * np.ones((1, NT))
    # K_l = 1 * np.ones(len(branch_data))  # Ampacity for each line
    # K_l = K_l[:, np.newaxis] * np.ones((1, NT))

    # ESS candidate to index
    # ESS_candidate = np.vectorize(bus_name_to_bus_index.get)(ESS_candidate).astype(int)
    ESS_candidate -= 1

    # summing all cost
    a = a_slack + a_PV + a_gen
    b = b_slack + b_PV + b_gen
    c = c_slack + c_PV + c_gen

    ###################################################################### Allocations constaints
    R_min = 0.05 / baseMVA * np.ones(N_bus)
    R_max = 4 / baseMVA * np.ones(N_bus)
    R_bounds = np.array([R_min,R_max])
    C_min = 0.1 / baseMVA * np.ones(N_bus)
    C_max = 7 /baseMVA * np.ones(N_bus)
    C_bounds = np.array([C_min,C_max])
    Fixed_cost = 100e3                  # CHF/unit suppose to be e3
    Power_rating_cost = 20000e3         # CHF/p.u. 
    Energy_capacity_cost = 30000e3      # CHF/p.u. 

    ################################################################# MILP
    Yp = 10

    # obj_2nd = np.zeros((NP,lim_iter)) # objective function result from the 2nd step (SOC-ACOPF) 
    # lambda_2nd = np.zeros((NP,N_bus,NT,lim_iter)) # dual variable lambda result from the 2nd step (SOC-ACOPF) 
    # mu_2nd = np.zeros((NP,N_bus,NT,lim_iter)) # dual variable mu result from the 2nd step (SOC-ACOPF) 
    previous_rating = np.zeros((N_bus,lim_iter)) # saved rating value from iter = iter-1
    previous_cap = np.zeros((N_bus,lim_iter)) # saved capacity value from iter = iter-1
    ESS_loc = np.zeros((N_bus,lim_iter))
    alpha_store = np.zeros(lim_iter)
    upperB_save = np.zeros(lim_iter)
    lowerB_save = np.zeros(lim_iter)
    fairness_price = np.zeros(lim_iter)
    invest_save = np.zeros(lim_iter)
    Elapsed_time = np.zeros(lim_iter)

    iter = 0
    convergence = 0
    R_div = 4
    obj_2nd = []              # list of list: obj_2nd[iter][m] = shape (NP,)
    lambda_2nd = []           # lambda_2nd[iter][m] = shape (NP, NB, NT)
    mu_2nd = []               # same shape

    while ((iter < lim_iter) & (convergence==0)):
        obj_iter = []
        lambda_iter = []
        mu_iter = []
        # master problem solving...
        obj_MP, invest_save[iter], ESS_loc[:,iter], Max_rating, Max_capacity, alpha_store[iter] = Allocation_2D(
            iter, NP, N_bus, ESS_candidate, R_bounds, C_bounds, 
            obj_2nd, lambda_2nd, mu_2nd, 
            previous_rating, previous_cap, 
            Fixed_cost, Power_rating_cost, Energy_capacity_cost
        )

        ################################################################## Battery Alocation 
        IndESS = np.where(ESS_loc[:,iter]==1)[0]

        # Charging limits
        ESS_cha_l = np.zeros((N_bus, NT))  # Lower charging limits
        ESS_cha_u = np.zeros((N_bus, NT))  # Upper charging limits
        ESS_cha_u[IndESS, :] = Max_rating[IndESS].reshape(-1, 1)  # Max charging limits
        # Discharging limits
        ESS_dis_l = np.zeros((N_bus, NT))  # Lower discharging limits
        ESS_dis_u = np.zeros((N_bus, NT))  # Upper discharging limits
        ESS_dis_u[IndESS, :] = Max_rating[IndESS].reshape(-1, 1)  # Max discharging limits

        # State of Charge (SoC)
        ESS_soc0 = np.zeros((N_bus))
        ESS_soc_l = np.zeros((N_bus, NT))  # Lower SoC limits
        ESS_soc_u = np.zeros((N_bus, NT))  # Upper SoC limits
        ESS_soc_u[IndESS, :] = Max_capacity[IndESS].reshape(-1, 1)   # Max SoC limits

        ESS_cha_bound = np.array([ESS_cha_l,ESS_cha_u])
        ESS_dis_bound = np.array([ESS_dis_l,ESS_dis_u])
        ESS_soc_bound = np.array([ESS_soc_l,ESS_soc_u])

        ###################################################################### 2nd stage SOC-ACOPF
        R_sample_size = int(np.ceil(NP / R_div))
        total_budget = NP
        M = max(1, int(total_budget / R_sample_size))

        for m in range(M):
            R_sample = np.random.choice(range(NP), size=R_sample_size, replace=False)
            obj_full, lambda_full, mu_full = run_SOC_ACOPF_for_samples(NP, R_sample, NT, baseMVA, N_bus, N_line, Yp, sending_node, receiving_node,
                                R_l, X_l, B_l, Pd, Qd, pn_bound, qn_bound, v_bound, G_n, B_n,
                                K_l, a, b, c, ESS_soc0, ESS_cha_bound, ESS_dis_bound, ESS_soc_bound,
                                Pn_solar_bound, freq_scenario)
            obj_iter.append(obj_full)
            lambda_iter.append(-lambda_full)
            mu_iter.append(-mu_full)
        obj_2nd.append(obj_iter)
        lambda_2nd.append(lambda_iter)
        mu_2nd.append(mu_iter)

        previous_rating[:,iter] = Max_rating
        previous_cap[:,iter] = Max_capacity

        ################################################################### Checking Convergence 
        confidence_level = 0.90
        Z = norm.ppf((1 + confidence_level) / 2)  # ~1.64

        W_sample = np.random.choice(range(NP), size=R_sample_size, replace=False)
        W = len(W_sample)
        obj_W, lambda_W, mu_W = run_SOC_ACOPF_for_samples(NP, W_sample, NT, baseMVA, N_bus, N_line, Yp, sending_node, receiving_node,
                    R_l, X_l, B_l, Pd, Qd, pn_bound, qn_bound, v_bound, G_n, B_n,
                    K_l, a, b, c, ESS_soc0, ESS_cha_bound, ESS_dis_bound, ESS_soc_bound,
                    Pn_solar_bound, freq_scenario)
        
        U_tilde = (NP / W) * np.sum(obj_W[W_sample]) + invest_save[iter] + fairness_price[iter]
        s_U = np.sqrt(1 / (W - 1) * np.sum((obj_W[W_sample] - np.mean(obj_W[W_sample])) ** 2))

        upper_confidence_bound = U_tilde + Z * s_U / np.sqrt(W)

        upperB_save[iter] = upper_confidence_bound
        lowerB_save[iter] = obj_MP

        confidence_adjusted_bound_gap = (upperB_save[iter] - lowerB_save[iter]) / upperB_save[iter]
        e_parameter = -0.0001

        if (confidence_adjusted_bound_gap < e_parameter) or (R_div == 1):
            print(f"Iteration {iter}, Gap: {abs(upperB_save[iter]-lowerB_save[iter])/upperB_save[iter]:.4f}")
            print("Upper bound :", upperB_save[iter])
            print("Lower bound :", lowerB_save[iter])
            print(f"[Outer Loop] Terminate the optimization process after {iter} outer iterations.")
            print(f"\t Termination Conditions: \n\t\tconfidence_adjusted_bound_gap = {confidence_adjusted_bound_gap}, R_div = {R_div}")
            print(f"\tupper_confidence_bound: {upperB_save[iter]}, lower_bound: {lowerB_save[iter]}")
            break

        print("---> NOT BREAKING ---")
        print(f"     upper_confidence_bound: {upper_confidence_bound}, lower_bound: {lowerB_save[iter]}, confidence_adjusted_bound_gap = {confidence_adjusted_bound_gap}")
        print(f"     Iter Count: {iter}")
        print("<--- ")

        # Continuing iteration
        iter+=1
        R_div = max(int(np.ceil(R_div / 1.5)), 1)

    total_time = dt.datetime.now() - start_time

    # Saving all files to analyze in another code
    res_path = 'res/single'
    os.makedirs(res_path, exist_ok=True)
    np.save(res_path+'/obj_2nd.npy', obj_2nd[:,:iter])
    np.save(res_path+'/lambda_2nd.npy', lambda_2nd[:,:,:,:iter])
    np.save(res_path+'/mu_2nd.npy', mu_2nd[:,:,:,:iter])
    np.save(res_path+'/previous_rating.npy', previous_rating[:,:iter])
    np.save(res_path+'/previous_cap.npy', previous_cap[:,:iter])
    np.save(res_path+'/ESS_loc.npy', ESS_loc[:,:iter])
    np.save(res_path+'/alpha_store.npy', alpha_store[:iter])
    np.save(res_path+'/upperB_save.npy', upperB_save[:iter])
    np.save(res_path+'/lowerB_save.npy', lowerB_save[:iter])
    np.save(res_path+'/invest_save.npy', invest_save[:iter])
    np.save(res_path+'/Elapsed_time.npy', Elapsed_time[:iter])
    np.save(res_path+'/total_time.npy', total_time)

### Newcut Function

In [ ]:
def Newcut(cb, cb_where):
    if cb_where == GRB.Callback.MIPSOL:
        global cutPoolLazy, cutPoolInfeas, cutPoolPartial
        global cutPoolLazy_added, z_last, gap_last, objval_last, bnd_last
        global nCuts, countLazy, R_sample, t_cur, objVal_cur, objVal_inner
        global z, t, z0, c, A, b, u, n, nR, R_div, method
        global cmcmd_prob, config, infeas_r

        if not cutPoolLazy_added:
            if len(cutPoolLazy) > 0:
                print(f"[Newcut] Adding {len(cutPoolLazy)} optimality lazy cuts from previous outer iterations")
                for lazyCut in cutPoolLazy:
                    if lazyCut.method == "scp_slim" and method == "scp_slim":
                        expr = lazyCut.offset[0] + sum(lazyCut.slope[i] * z[i] for i in range(len(z)))
                        cb.cbLazy(t >= expr)
            
            if len(cutPoolInfeas) > 0:
                print(f"[Newcut] Adding {len(cutPoolInfeas)} infeasibility lazy cuts from previous outer iterations")
                for c_cut in cutPoolInfeas:
                    for r in range(c_cut.b.shape[1]):
                        if (np.any(c_cut.b[:,r] > 0) or np.any(c_cut.p[:,:,r] != 0)):
                            r_tilde = c_cut.R_sample[r]
                            lhs = np.sum(c_cut.p[:,:,r] * b[:,:,r_tilde])
                            rhs = sum(z[e] * u[e] * c_cut.b[e, r] for e in range(n))
                            cb.cbLazy(lhs <= rhs)
            
            cutPoolLazy_added = True

        z_cur = np.zeros(len(z0))
        for i in range(n):
            z_cur[i] = cb.cbGetSolution(z[i])

        if config.round_z0:
            z_cur = np.round(z_cur, decimals=3)

        primal_bound = cb.cbGet(GRB.Callback.MIPSOL_OBJBST)
        dual_bound = cb.cbGet(GRB.Callback.MIPSOL_OBJBND)
        objbst = primal_bound
        objbnd = dual_bound
        mip_gap = min(gapcalculate(objbst, objbnd), 1)

        node_count = cb.cbGet(GRB.Callback.MIPSOL_NODCNT)

        if node_count >= 0:
            is_zcur_feasible = cMCMD_is_feasible(A, b, z_cur, u)
            print(f" [Newcut] Status : {node_count} --- Saving z_last\n\t\tis_zcur_feasible = {is_zcur_feasible} --- ({np.sum(z_cur)}/{len(z_cur)})")
            if is_zcur_feasible:
                z_last = z_cur.copy()
                gap_last = mip_gap
                objval_last = objbst
                bnd_last = max(objbnd, bnd_last)

        R_sample = np.sort(np.random.choice(range(nR), size=int(np.ceil(nR/R_div)), replace=False))
        print(f"\t\t\tR_sample : {R_sample}")

        add_infeasibility_cut = False
        if method == "scp_slim":
            nCuts += 1
            
            print(" [Newcut] Calculating Cut")
            obj, grad_obj, feas_status, _ = cMCMDCutting_plane(
                cmcmd_prob, config, z_cur, R_sample=R_sample
            )
            add_infeasibility_cut = np.any(np.isnan(obj))
            
            if add_infeasibility_cut and config.use_partial_cuts:
                R_sample_partial = np.setdiff1d(R_sample, R_sample[infeas_r])
                if len(R_sample_partial) >= 1:
                    print(f"\t\t[use_partial_cuts] Adding {len(R_sample_partial)} optimality cuts for infeasible z")
                    obj_partial, grad_obj_partial, feas_status = cMCMDCutting_plane(
                        cmcmd_prob, config, z_cur, R_sample=R_sample_partial
                    )
                    if np.isnan(obj_partial):
                        raise ValueError("\t\t\t[use_partial_cuts] NaN obj_partial")
                    else:
                        expr = obj_partial + sum(grad_obj_partial[i] * (z[i] - z_cur[i]) for i in range(n))
                        cb.cbLazy(t >= expr)

                        s0 = type.PrimalSolution(
                            np.where(z_cur > 0)[0].tolist(), 
                            z_cur.tolist(), 
                            obj_partial + np.dot(c, z_cur),
                            [obj_partial - np.dot(grad_obj_partial, z_cur[:n])],
                            grad_obj_partial, 
                            False, 
                            method, 
                            R_sample.tolist(), 
                            [], 
                            {}
                        )
                        cutPoolPartial.append(s0)
            
            if not add_infeasibility_cut:
                expr = obj + sum(grad_obj[i] * (z[i] - z_cur[i]) for i in range(n))
                cb.cbLazy(t >= expr)
                print("\t[Newcut] Adding Slim Cut")

                s0 = PrimalSolution(
                    np.where(z_cur > 0)[0].tolist(), 
                    z_cur.tolist(),
                    obj + np.dot(c, z_cur),
                    [obj - np.dot(grad_obj, z_cur[:n])],
                    grad_obj, 
                    False, 
                    method, 
                    R_sample.tolist(), 
                    {}
                )
                cutPoolLazy.append(s0)
                countLazy += 1
                if countLazy % 10 == 0:
                    print(f"countlazy = {countLazy}")
        
        if add_infeasibility_cut:
            p_certificate, beta_certificate, infeas_r = get_infeas_certificate(cmcmd_prob, z_cur, R_sample=R_sample)
            print(f"[Newcut] Adding Infeasibility Cut\n\t\tR_sample = {R_sample}\n\t\tinfeas_r = {R_sample[infeas_r]}")

            if not (np.any(np.isnan(p_certificate)) and np.any(np.isnan(beta_certificate))):
                for r in infeas_r:
                    if (np.any(beta_certificate[:,r] > 0) or np.any(p_certificate[:,:,r] != 0)):
                        r_tilde = R_sample[r]
                        LHS_check = np.sum(p_certificate[:,:,r] * b[:,:,r_tilde]) - sum(z_cur[e] * u[e] * beta_certificate[e, r] for e in range(n))
                        print(f"\t\t\tAdding cut for r = {r} (r_tilde = {r_tilde}) [LHS_check at z_cur = {LHS_check}]")

                        lhs = np.sum(p_certificate[:,:,r] * b[:,:,r_tilde])
                        rhs = sum(z[e] * u[e] * beta_certificate[e, r] for e in range(n))
                        cb.cbLazy(lhs <= rhs)

                infeas_cut = type.InfeasibleCut(
                    z_cur.tolist(), 
                    R_sample.tolist(), 
                    p_certificate, 
                    beta_certificate, 
                    infeas_r
                )
                cutPoolInfeas.append(infeas_cut)
        
        if method == "scp_slim":
            t_cur = cb.cbGetSolution(t)

            objVal_cur = np.dot(c, z_cur) + t_cur
            objVal_inner = np.dot(c, z_cur) + obj

            print(f"[Newcut End Logging]\n  --objbst = {objbst}\n  --objbnd = {objbnd}\n  --mip_gap = {mip_gap}\n  --objVal_cur = {objVal_cur}\n  --objVal_inner = {objVal_inner}")